In [3]:
import os
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tempfile import TemporaryDirectory

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split

import torchvision
from torchvision import datasets, models, transforms

import wandb

cudnn.benchmark = True
plt.ion()

!wandb login b199ff6d9c0a9f7fc2901490f41b4dacc0ef21d6


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
# Customed Dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = self._find_classes()
        self.image_paths, self.labels = self._load_data()

    def _find_classes(self):
        classes = sorted([d for d in os.listdir(self.data_dir) if os.path.isdir(os.path.join(self.data_dir, d))])
        return classes

    def _load_data(self):
        image_paths = []
        labels = []
        for label in self.classes:
            class_dir = os.path.join(self.data_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                image_paths.append(img_path)
                labels.append(int(label))
        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        label = int(label)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
# Parameters

# Change batchsize to fit hardware
batch_size = 128

# Training parameters
num_classes = 2139  
learning_rate = 0.0001
num_epochs = 30

# Scheduler
step_size = 5
gamma = 1


In [6]:
# Location of data
train_dir = '/kaggle/input/wb-recognition-dataset/wb_recognition_dataset/train'
val_dir = '/kaggle/input/wb-recognition-dataset/wb_recognition_dataset/val'

In [7]:
# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [8]:
# Datasets from each folder
image_datasets = {
    'train': CustomDataset(train_dir, data_transforms['train']),
    'val': CustomDataset(val_dir, data_transforms['val']),
}

# Dataloader iterators
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4, pin_memory = True),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory = True),
}

# Size of datasets
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [9]:
# Number of images and labels
print('Number of images in train: ', dataset_sizes['train'])
print('Number of labels in train: ',len(image_datasets['train'].classes))
print('Number of images in val: ', dataset_sizes['val'])
print('Number of labels in val: ', len(image_datasets['val'].classes))

Number of images in train:  56813
Number of labels in train:  2130
Number of images in val:  1392
Number of labels in val:  595


In [10]:
#VGG16
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False
n_inputs = model.classifier[6].in_features

# Add on classifier
model.classifier[6] = nn.Sequential(
nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2), nn.Linear(256, 2139), nn.LogSoftmax(dim=1))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 166MB/s]  


In [11]:
# Training loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Decay the learning rate by 10% every 5 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size= step_size, gamma=gamma)

In [12]:
# Move to gpu 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [13]:
# Check model
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
def train_model(model, criterion, optimizer, dataloaders, model_last_pth, model_best_pth, num_epochs):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        dataloaders (PyTorch dataloader): dataloaders to iterate through
        model_last_pth, model_best_pth (str ending in '.pt'): file path to save the model state dict
        num_epochs (int): maximum number of training epochs

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
    """
    
    # Min validation loss
    valid_loss_min = np.Inf
    
    # Main loop
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} / {num_epochs}')
        print('-' * 10)
        
        # Go through training and validation phase each epoch
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            # Keep track of loss and corrects each epoch
            running_loss = 0.0
            running_corrects = 0

            # Training loop
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Clear gradients
                optimizer.zero_grad()
                
                # Predicted outputs and loss of gradients
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backpropagation and update parameters
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Update loss and number of correct predictions
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            # Step the scheduler if in training phase
            if phase == 'train':
                scheduler.step()
            
            # Calculate loss and accuracy of each epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            
            if phase == 'val':
                if epoch_loss < valid_loss_min:
                    valid_loss_min = epoch_loss
                    torch.save(model.state_dict(), model_best_pth)
            
            print(f'{phase}\t Loss: {epoch_loss:.4f}\t Accuracy: {epoch_acc:.4f}')
            
            wandb.log({f'{phase}_loss': epoch_loss, f'{phase}_acc': epoch_acc})
        
        # Save model every epoch
        torch.save(model.state_dict(), model_last_pth)

    return model

In [18]:
# Path to save model (last and best)
model_last_pth = '/kaggle/working/VGG16-imagenet-2-last.pt'
model_best_pth = '/kaggle/working/VGG16-imagenet-2-best.pt'

# Saved model 
saved_model_path = '/kaggle/input/vgg16/pytorch/vgg16_v1/1/vgg16-transfer-imagenet-normalization-0.pt'

In [19]:
wandb.init(project='ImageProcessing-project', sync_tensorboard=True)

# Load saved model
try:
    model.load_state_dict(torch.load(saved_model_path))
    print('Loaded saved model successfully')
except FileNotFoundError:
    print('File not found')
except Exception as e:
    print(f'An error occurred: {e}')

print()

model.to(device)
    
model = train_model(model, criterion, optimizer, dataloaders, model_last_pth, model_best_pth, num_epochs)

wandb.finish()

An error occurred: PytorchStreamReader failed locating file data/3: file not found

Epoch 1 / 30
----------
train	 Loss: 4.8253	 Accuracy: 0.1294
val	 Loss: 3.8262	 Accuracy: 0.3326
Epoch 2 / 30
----------
train	 Loss: 4.8077	 Accuracy: 0.1328
val	 Loss: 3.8013	 Accuracy: 0.3341
Epoch 3 / 30
----------
train	 Loss: 4.7864	 Accuracy: 0.1350
val	 Loss: 3.7999	 Accuracy: 0.3362
Epoch 4 / 30
----------
train	 Loss: 4.7758	 Accuracy: 0.1354
val	 Loss: 3.7823	 Accuracy: 0.3376
Epoch 5 / 30
----------
train	 Loss: 4.7590	 Accuracy: 0.1399
val	 Loss: 3.7982	 Accuracy: 0.3333
Epoch 6 / 30
----------
train	 Loss: 4.7487	 Accuracy: 0.1384
val	 Loss: 3.7838	 Accuracy: 0.3348
Epoch 7 / 30
----------
train	 Loss: 4.7293	 Accuracy: 0.1410
val	 Loss: 3.7737	 Accuracy: 0.3434
Epoch 8 / 30
----------
train	 Loss: 4.7119	 Accuracy: 0.1408
val	 Loss: 3.7653	 Accuracy: 0.3341
Epoch 9 / 30
----------
train	 Loss: 4.6980	 Accuracy: 0.1428
val	 Loss: 3.7536	 Accuracy: 0.3333
Epoch 10 / 30
----------
train	 Lo

KeyboardInterrupt: 